In [2]:
%%capture
import random
import math
import string
import numpy as np
from utils import *

In [3]:
CERT_AUTH = { 'e': 65537, 'N': 4779050428557429001097376236264073698001638047110453239392250614634445097932065145325834667638722969517681966380215207854983322387622144528819783968142689}

In [4]:
class Client:
    def __init__(self, keybits=512):
        rsa_keypair = generateRSAKeypair(keybits)
        
        self.RSA_PUBLIC_KEY = (rsa_keypair[0], rsa_keypair[1])
        self.RSA_PRIVATE_KEY = (rsa_keypair[2])
        
    def signMessage(self, message):
        return signRSA(self.RSA_PRIVATE_KEY, message, self.RSA_PUBLIC_KEY[1])
        
    def verifyServerCertificate(self, serverPublicKey, signature):
        """
        Asserts the server's certificate is correctly signed by the given certificate authority.

        The certificate is on int(e||N).
        """
        
        e, N = CERT_AUTH['e'], CERT_AUTH['N']

        if not verifyRSA(e, signature, N, int(str(serverPublicKey[0]) + str(serverPublicKey[1]))):
            return False
        
        self.SERVER_PUBLIC_KEY = serverPublicKey
        
        return True
    
    def verifyECDHParameters(self, ellipticCurve, N, P, orderP, signature):
        intToSign = int(str(ellipticCurve[0]) + str(ellipticCurve[1]) + str(N) + str(P[0]) + str(P[1]))
        
        if not verifyRSA(self.SERVER_PUBLIC_KEY[0], signature, self.SERVER_PUBLIC_KEY[1], intToSign):
            return False
        
        self.ELLIPTIC_CURVE = ellipticCurve
        self.ELLIPTIC_CURVE_MODULUS = N
        self.GENERATOR_POINT = P
        self.GENERATOR_POINT_ORDER = orderP
        
        return True
    
    
    def generateSignedECDHMessage(self):
        """
        Assumes we have already setup a valid connection.
        """
        
        nP, n = generateECDH(self.GENERATOR_POINT, self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS, self.GENERATOR_POINT_ORDER)
        
        intToSign = int(str(nP[0]) + str(nP[1]))
        
        self.ECDH_SECRET = n
        
        return nP, self.signMessage(intToSign)
    
    def verifyECDHMessage(self, P, signature, publicKey):
        """
        Verifies point P was signed by the given publicKey (e, N).
        """

        expected = int(str(P[0]) + str(P[1]))

        return verifyRSA(publicKey[0], signature, publicKey[1], expected)
    
    def acceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        
        assert self.verifyECDHMessage(P, signature, self.SERVER_PUBLIC_KEY), "Invalid signature on received ECDH message."
        
        # Signature verified
        sharedSecretPoint = doubleAndAdd(P, self.ECDH_SECRET, self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS)
        
        self.SHARED_SECRET_INT = pointToMessage(sharedSecretPoint)
        
        return True
    
    def encryptMessage(self, message):
        """
        Given string message, encrypts with the current setup
        """
        
        return encryptAES(self.SHARED_SECRET_INT, bytes(message, 'utf-8'))
    
    def decryptMessage(self, encrypted):
        """
        Given encrypted message, decrypts with the current setup
        """
        
        return decryptAES(self.SHARED_SECRET_INT, encrypted)
    
    def generateRequest(self, obj):
        encStr = self.encryptMessage(json.dumps(obj))
        
        hmac = generateHMAC(self.SHARED_SECRET_INT, bytes(encStr, 'utf-8'))
        
        requestObj = {'data': encStr, 'hmac': hmac}
        
        return requestObj